In [ ]:
# 拆分pdf（每5页为一份）

import os
from PyPDF2 import PdfReader, PdfWriter
from PyPDF2.errors import PdfReadError

def split_pdf_by_chunk(input_pdf_path, output_folder, chunk_size=5):
    os.makedirs(output_folder, exist_ok=True)

    try:
        reader = PdfReader(input_pdf_path)
    except PdfReadError as e:
        print(f"❌ 无法读取 PDF：{input_pdf_path}，错误信息：{e}")
        return
    except Exception as e:
        print(f"❌ 未知错误读取 PDF：{input_pdf_path}，错误信息：{e}")
        return

    total_pages = len(reader.pages)
    base_name = os.path.splitext(os.path.basename(input_pdf_path))[0]  # 获取原始文件名（不带扩展名）

    for start in range(0, total_pages, chunk_size):
        writer = PdfWriter()
        end = min(start + chunk_size, total_pages)
        for i in range(start, end):
            writer.add_page(reader.pages[i])

        part_num = start // chunk_size + 1
        output_path = os.path.join(output_folder, f"{base_name}_part_{part_num}.pdf")  # 修改命名方式
        with open(output_path, "wb") as f_out:
            writer.write(f_out)

        print(f"✅ 已保存：{output_path}")
    
    print(f"🎉 完成：{input_pdf_path} 的拆分，共 {total_pages} 页，每 {chunk_size} 页一组。")


def split_all_pdfs_in_folder(input_folder, output_base_folder, chunk_size=5):
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(".pdf") and not filename.startswith("._"):
            input_pdf_path = os.path.join(input_folder, filename)
            output_folder = os.path.join(output_base_folder, os.path.splitext(filename)[0])  # 输出文件夹基于原始文件名

            try:
                split_pdf_by_chunk(input_pdf_path, output_folder, chunk_size)
            except Exception as e:
                print(f"⚠️ 跳过文件：{filename}，出错原因：{e}")


# 示例用法
if __name__ == "__main__":
    input_folder = r"Your_pdfs_Path"  # 原始PDF文件夹路径
    output_base_folder = r"Your_split_pdfs_Path"  # 拆分后保存的根目录
    chunk_size = 5  # 每多少页为一组
    split_all_pdfs_in_folder(input_folder, output_base_folder, chunk_size)
